In [391]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML4VA/community-policing-data.csv')
df.head()

<ipython-input-391-32cf455b945e>:3: DtypeWarning: Columns (5,10,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML4VA/community-policing-data.csv')


,STOP_DATE,AGENCY NAME,LOCATION,JURISDICTION,REASON FOR STOP,PERSON TYPE,RACE,ETHNICITY,AGE,GENDER,...,ACTION TAKEN,SPECIFIC VIOLATION,VIRGINIA CRIME CODE,PERSON SEARCHED,VEHICLE SEARCHED,ADDITIONAL ARREST,FORCE USED BY OFFICER,FORCE USED BY SUBJECT,RESIDENCY,VIOLATION TYPE
0,7/1/2020,Broadway Police Department,S MAIN/FORTH ST,ROCKINGHAM CO,TRAFFIC VIOLATION,NaN,WHITE,NOT HISPANIC OR LATINO,18.0,MALE,...,WARNING ISSUED,46.2-833,NaN,NO,NO,NO,NaN,NaN,NaN,COMMONWEALTH
1,7/1/2020,Isle Of Wight County Sheriff's Office,36.967676 -76.511763,ISLE OF WIGHT CO,TRAFFIC VIOLATION,NaN,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,26.0,MALE,...,WARNING ISSUED,46.2-804,NaN,NO,NO,NO,NaN,NaN,NaN,COMMONWEALTH
2,7/1/2020,Isle Of Wight County Sheriff's Office,36.95375 -76.540314,ISLE OF WIGHT CO,EQUIPMENT VIOLATION,NaN,UNKNOWN,UNKNOWN,82.0,NaN,...,WARNING ISSUED,46.2-1157,NaN,NO,NO,NO,NaN,NaN,NaN,COMMONWEALTH
3,7/1/2020,Isle Of Wight County Sheriff's Office,36.969732 -76.569873,ISLE OF WIGHT CO,EQUIPMENT VIOLATION,NaN,WHITE,UNKNOWN,61.0,MALE,...,WARNING ISSUED,46.2-1157,NaN,NO,NO,NO,NaN,NaN,NaN,COMMONWEALTH
4,7/1/2020,Isle Of Wight County Sheriff's Office,36.949132 -76.596691,ISLE OF WIGHT CO,TRAFFIC VIOLATION,NaN,WHITE,NOT HISPANIC OR LATINO,37.0,MALE,...,CITATION/SUMMONS,46.2-1157,NaN,NO,NO,NO,NaN,NaN,NaN,COMMONWEALTH


In [392]:
print(df.shape)

(3371015, 21)


In [393]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3371015 entries, 0 to 3371014
Data columns (total 21 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   STOP_DATE              object 
 1   AGENCY NAME            object 
 2   LOCATION               object 
 3   JURISDICTION           object 
 4   REASON FOR STOP        object 
 5   PERSON TYPE            object 
 6   RACE                   object 
 7   ETHNICITY              object 
 8   AGE                    float64
 9   GENDER                 object 
 10  ENGLISH SPEAKING       object 
 11  ACTION TAKEN           object 
 12  SPECIFIC VIOLATION     object 
 13  VIRGINIA CRIME CODE    object 
 14  PERSON SEARCHED        object 
 15  VEHICLE SEARCHED       object 
 16  ADDITIONAL ARREST      object 
 17  FORCE USED BY OFFICER  object 
 18  FORCE USED BY SUBJECT  object 
 19  RESIDENCY              object 
 20  VIOLATION TYPE         object 
dtypes: float64(1), object(20)
memory usage: 540.1+ MB


In [394]:
missing_values = df.isnull().sum()
print(missing_values)

STOP_DATE                      0
AGENCY NAME                    0
LOCATION                    4963
JURISDICTION                   8
REASON FOR STOP            22366
PERSON TYPE               958398
RACE                          15
ETHNICITY                     22
AGE                           56
GENDER                      2906
ENGLISH SPEAKING          966913
ACTION TAKEN                4214
SPECIFIC VIOLATION         73011
VIRGINIA CRIME CODE      2052157
PERSON SEARCHED            24128
VEHICLE SEARCHED           28504
ADDITIONAL ARREST        2442425
FORCE USED BY OFFICER     966413
FORCE USED BY SUBJECT     966431
RESIDENCY                1474357
VIOLATION TYPE           2427194
dtype: int64


In [395]:
df["PERSON TYPE"].value_counts()

,count
PERSON TYPE,
DRIVER,2327699
OTHER INDIVIDUAL,68598
PASSENGER,16320


In [396]:
df["ENGLISH SPEAKING"].value_counts()

,count
ENGLISH SPEAKING,
YES,2331202
NO,72899
44108.47767,1


In [397]:
df["VIRGINIA CRIME CODE"].value_counts()

,count
VIRGINIA CRIME CODE,
MOV6450I9,181162
MOV6481I9,128023
REG6695S9,90234
MOV6449I9,71145
SIG6552I9,66283
...,...
BUR2215T2,1
LAR2383C9,1
WPN5298A6,1


In [398]:
df["ADDITIONAL ARREST"].value_counts()

,count
ADDITIONAL ARREST,
NO,911238
YES,17352


In [399]:
df["RESIDENCY"].value_counts()

,count
RESIDENCY,
RESIDENT OF CITY/COUNTY OF STOP,809761
OTHER VIRGINIA JURISDICTION RESIDENT,610906
OUT OF STATE RESIDENT,412658
UNKNOWN,63333


In [400]:
df["VIOLATION TYPE"].value_counts()

,count
VIOLATION TYPE,
COMMONWEALTH,740080
LOCAL,199163
Commonwealth,4234
Local,344


In [401]:
# create a copy of df into df2
df2 = df.copy()
# drop columns from df2
# df2 = df2.drop('PERSON TYPE', axis=1)
# df2 = df2.drop('ENGLISH SPEAKING', axis=1)
# df2 = df2.drop('VIRGINIA CRIME CODE', axis=1)
df2 = df2.drop('ADDITIONAL ARREST', axis=1)
# df2 = df2.drop('RESIDENCY', axis=1)
df2 = df2.drop('VIOLATION TYPE', axis=1)
df = df.drop('VIRGINIA CRIME CODE', axis=1)
print(df.columns.tolist())
print(df2.columns.tolist())

['STOP_DATE', 'AGENCY NAME', 'LOCATION', 'JURISDICTION', 'REASON FOR STOP', 'PERSON TYPE', 'RACE', 'ETHNICITY', 'AGE', 'GENDER', 'ENGLISH SPEAKING', 'ACTION TAKEN', 'SPECIFIC VIOLATION', 'PERSON SEARCHED', 'VEHICLE SEARCHED', 'ADDITIONAL ARREST', 'FORCE USED BY OFFICER', 'FORCE USED BY SUBJECT', 'RESIDENCY', 'VIOLATION TYPE']
['STOP_DATE', 'AGENCY NAME', 'LOCATION', 'JURISDICTION', 'REASON FOR STOP', 'PERSON TYPE', 'RACE', 'ETHNICITY', 'AGE', 'GENDER', 'ENGLISH SPEAKING', 'ACTION TAKEN', 'SPECIFIC VIOLATION', 'VIRGINIA CRIME CODE', 'PERSON SEARCHED', 'VEHICLE SEARCHED', 'FORCE USED BY OFFICER', 'FORCE USED BY SUBJECT', 'RESIDENCY']


In [402]:
print(len(df.columns.tolist()))
print(len(df2.columns.tolist()))

20
19


In [403]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3371015 entries, 0 to 3371014
Data columns (total 20 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   STOP_DATE              object 
 1   AGENCY NAME            object 
 2   LOCATION               object 
 3   JURISDICTION           object 
 4   REASON FOR STOP        object 
 5   PERSON TYPE            object 
 6   RACE                   object 
 7   ETHNICITY              object 
 8   AGE                    float64
 9   GENDER                 object 
 10  ENGLISH SPEAKING       object 
 11  ACTION TAKEN           object 
 12  SPECIFIC VIOLATION     object 
 13  PERSON SEARCHED        object 
 14  VEHICLE SEARCHED       object 
 15  ADDITIONAL ARREST      object 
 16  FORCE USED BY OFFICER  object 
 17  FORCE USED BY SUBJECT  object 
 18  RESIDENCY              object 
 19  VIOLATION TYPE         object 
dtypes: float64(1), object(19)
memory usage: 514.4+ MB


In [404]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3371015 entries, 0 to 3371014
Data columns (total 19 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   STOP_DATE              object 
 1   AGENCY NAME            object 
 2   LOCATION               object 
 3   JURISDICTION           object 
 4   REASON FOR STOP        object 
 5   PERSON TYPE            object 
 6   RACE                   object 
 7   ETHNICITY              object 
 8   AGE                    float64
 9   GENDER                 object 
 10  ENGLISH SPEAKING       object 
 11  ACTION TAKEN           object 
 12  SPECIFIC VIOLATION     object 
 13  VIRGINIA CRIME CODE    object 
 14  PERSON SEARCHED        object 
 15  VEHICLE SEARCHED       object 
 16  FORCE USED BY OFFICER  object 
 17  FORCE USED BY SUBJECT  object 
 18  RESIDENCY              object 
dtypes: float64(1), object(18)
memory usage: 488.7+ MB


In [405]:
# # loop through every row in df
# count = 0
# for i in range(df.shape[0]):  # Loop over rows
#     filledIn = True
#     for j in range(df.shape[1]):  # Loop over columns
#         if pd.isnull(df.iloc[i, j]):
#             hasMissing = False
#             break
#     if filledIn:
#         count += 1
# print(count)

In [406]:
df_cleaned = df.dropna(axis=0, how='any', inplace=False)
df2_cleaned = df2.dropna(axis=0, how='any', inplace=False)

print(df_cleaned.shape)
print(df2_cleaned.shape)

(0, 20)
(1022394, 19)


In [407]:
df2_cleaned.head(10)

,STOP_DATE,AGENCY NAME,LOCATION,JURISDICTION,REASON FOR STOP,PERSON TYPE,RACE,ETHNICITY,AGE,GENDER,ENGLISH SPEAKING,ACTION TAKEN,SPECIFIC VIOLATION,VIRGINIA CRIME CODE,PERSON SEARCHED,VEHICLE SEARCHED,FORCE USED BY OFFICER,FORCE USED BY SUBJECT,RESIDENCY
207831,9/9/2021,Fairfax County Police Department,6221 ROLLING RD,FAIRFAX CO,TERRY STOP,DRIVER,WHITE,HISPANIC OR LATINO,64.0,FEMALE,YES,ARREST,18.2-57(A),ASL1313M1,YES,NO,NO,NO,OTHER VIRGINIA JURISDICTION RESIDENT
207832,8/8/2021,Fairfax County Police Department,BRADDOCK RD,FAIRFAX CO,TRAFFIC VIOLATION,DRIVER,WHITE,UNKNOWN,25.0,FEMALE,YES,ARREST,18.2-266,DWI5413M1,YES,NO,NO,NO,RESIDENT OF CITY/COUNTY OF STOP
207833,9/9/2021,Fairfax County Police Department,5885 KINGSTOWNE BLVD,FAIRFAX CO,CALLS FOR SERVICE,OTHER INDIVIDUAL,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,33.0,MALE,YES,ARREST,18.2-103,LAR2339F6,YES,NO,NO,NO,OTHER VIRGINIA JURISDICTION RESIDENT
207834,9/9/2021,Fairfax County Police Department,1645 WASHINGTON PLZ,FAIRFAX CO,TERRY STOP,OTHER INDIVIDUAL,WHITE,HISPANIC OR LATINO,25.0,MALE,YES,ARREST,18.2-250(A)(b1),NAR3024M2,NO,NO,NO,NO,RESIDENT OF CITY/COUNTY OF STOP
207835,9/9/2021,Fairfax County Police Department,6418 QUANDER RD,FAIRFAX CO,CALLS FOR SERVICE,DRIVER,WHITE,NOT HISPANIC OR LATINO,54.0,FEMALE,YES,ARREST,18.2-266,DWI5434M1,YES,NO,NO,NO,OTHER VIRGINIA JURISDICTION RESIDENT
207836,9/9/2021,Fairfax County Police Department,1961 CHAIN BRIDGE RD,FAIRFAX CO,CALLS FOR SERVICE,OTHER INDIVIDUAL,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,22.0,FEMALE,YES,ARREST,18.2-96(2),LAR2369F6,NO,NO,NO,NO,OUT OF STATE RESIDENT
207838,9/10/2021,Fairfax County Police Department,2513 RESTON PKWY,FAIRFAX CO,TRAFFIC VIOLATION,DRIVER,WHITE,NOT HISPANIC OR LATINO,42.0,FEMALE,YES,ARREST,18.2-266,DWI5443M1,YES,NO,NO,NO,RESIDENT OF CITY/COUNTY OF STOP
207840,9/10/2021,Fairfax County Police Department,7220 TELEGRAPH SQUARE DR,FAIRFAX CO,CALLS FOR SERVICE,OTHER INDIVIDUAL,WHITE,HISPANIC OR LATINO,50.0,MALE,YES,ARREST,46.2-852,REC6626F6,NO,NO,NO,NO,RESIDENT OF CITY/COUNTY OF STOP
207841,9/10/2021,Fairfax County Police Department,1961 CHAIN BRIDGE RD,FAIRFAX CO,CALLS FOR SERVICE,DRIVER,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,36.0,MALE,YES,ARREST,18.2-96(1),LAR2369F6,NO,NO,NO,NO,OTHER VIRGINIA JURISDICTION RESIDENT
207842,9/10/2021,Fairfax County Police Department,7910 RICHMOND HWY,FAIRFAX CO,CALLS FOR SERVICE,OTHER INDIVIDUAL,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,26.0,FEMALE,YES,ARREST,18.2-96(2),LAR2366M1,NO,NO,NO,NO,OUT OF STATE RESIDENT


In [408]:
# # only return "SPECIFIC VIOLATION" value counts with less than 50,000

# value_counts = df2_cleaned["SPECIFIC VIOLATION"].value_counts()

# # Filter for counts less than 50,000
# filtered_counts = value_counts[value_counts < 2000]

# # Display the result
# print(filtered_counts)


# # df2_cleaned["SPECIFIC VIOLATION"].value_counts()

In [409]:
from scipy.stats import chi2_contingency

# Cramér's V function
def cramers_v(x, y):
    contingency_table = pd.crosstab(x, y)
    chi2, _, _, _ = chi2_contingency(contingency_table)
    n = contingency_table.sum().sum()
    min_dim = min(contingency_table.shape) - 1
    return np.sqrt(chi2 / (n * min_dim))

# Get all feature columns
features = df2_cleaned.columns.tolist()

# List to store correlations
corr = []

# Iterate through features and calculate Cramér's V for each
for feature in features:
    if feature != "ACTION TAKEN":  # Avoid comparing the target variable with itself
        corr_value = cramers_v(df2_cleaned[feature], df2_cleaned["ACTION TAKEN"])  # Pass the actual data
        corr.append((feature, corr_value))  # Append as a tuple (feature name, correlation)

# Sort the results by correlation value (descending order)
corr.sort(key=lambda x: x[1], reverse=True)

# Print the sorted correlations
for feature, value in corr:
    print(f"{feature}: {value:.4f}")

LOCATION: 0.7698
PERSON SEARCHED: 0.6413
SPECIFIC VIOLATION: 0.6003
VIRGINIA CRIME CODE: 0.5702
PERSON TYPE: 0.4845
REASON FOR STOP: 0.4138
AGENCY NAME: 0.2993
JURISDICTION: 0.2711
VEHICLE SEARCHED: 0.1630
FORCE USED BY OFFICER: 0.1351
FORCE USED BY SUBJECT: 0.1188
AGE: 0.1163
STOP_DATE: 0.1001
GENDER: 0.0750
RESIDENCY: 0.0730
ENGLISH SPEAKING: 0.0694
ETHNICITY: 0.0626
RACE: 0.0541


In [410]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Downsample the entire dataset to 100,000 rows
df, _ = train_test_split(
    df2_cleaned,
    train_size=100000,  # Specify the total size
    stratify=df2_cleaned['ACTION TAKEN'],  # Stratify on the target variable
    random_state=42
)

Total rows in dataset: 100000
Training set size: (80000, 80000)
Test set size: (20000, 20000)


START JACOB'S LINEAR REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.model_selection import GridSearchCV

label_encoder = LabelEncoder()
df['ACTION TAKEN'] = label_encoder.fit_transform(df['ACTION TAKEN'])

X = df.drop(columns=['ACTION TAKEN'])
y = df['ACTION TAKEN']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(exclude=['object']).column

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('convert_to_str', FunctionTransformer(lambda x: x.astype(str), validate=False)),
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=500))
])

In [ ]:
param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10, 100]
}

# Set up GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Output the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# Make predictions using the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Convert predictions and y_test back to original categorical labels
y_pred_labels = label_encoder.inverse_transform(y_pred)
y_test_labels = label_encoder.inverse_transform(y_test)

# Evaluate and display results
accuracy = accuracy_score(y_test_labels, y_pred_labels)
print(f"Test Accuracy: {accuracy}")
print("\nClassification Report:\n", classification_report(y_test_labels, y_pred_labels, target_names=label_encoder.classes_))

START HENRY'S SVM

In [ ]:
categorical_columns = df.select_dtypes(include=['object']).columns
df[categorical_columns] = df[categorical_columns].astype(str)

# Convert the target column to numeric labels
label_encoder = LabelEncoder()
df['ACTION TAKEN'] = label_encoder.fit_transform(df['ACTION TAKEN'])

# Separate features and target
X = df.drop(columns=['ACTION TAKEN'])  # Replace 'ACTION TAKEN' with your actual target column name
y = df['ACTION TAKEN']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=27
)

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(exclude=['object']).columns

# Define preprocessing for numeric and categorical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Convert all categorical columns to string before processing
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

X_train_prepared = preprocessor.fit_transform(X_train)
X_test_prepared = preprocessor.transform(X_test)

# Check sizes to verify
print(f"Total rows in dataset: {len(df)}")  # Should be 100,000
print(f"Training set size: {len(X_train), len(y_train)}")
print(f"Test set size: {len(X_test), len(y_test)}")

In [411]:
X_train_prepared.shape

(80000, 45978)

In [412]:
X_test_prepared.shape

(20000, 45978)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Define a parameter grid, using 'auto' and 'scale' options for gamma and different C values
param_grid = {
    'C': [1],  # Experiment with a range of C values
    'gamma': ['auto'],
}

# Create an SVM model
svm_clf = SVC(kernel='rbf', decision_function_shape='ovr', random_state=27)

# Use GridSearchCV with parallel processing (e.g., using all cores)
grid_search = GridSearchCV(svm_clf, param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_prepared, y_train)

# Get the best model from the grid search
best_svm_clf = grid_search.best_estimator_

# Predict on the test data
y_pred = best_svm_clf.predict(X_test_prepared)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

print("Best Parameters:", grid_search.best_params_)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", report)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
